# $sortByCount 的操作

In [ ]:
# find one company document
db.companies.findOne()

# create text index
db.companies.createIndex({ description: "text", overview: "text" })

In [ ]:
# find companies matching term `networking` using text search
db.companies.aggregate([
    {
        $match: {
            $text: { $search: "network" }
        }
    }
]).pretty()

In [ ]:
db.companies.aggregate([
    {
        $match: {
            $text: { $search: "network" }
        }
    },
    {
        $group: {
            _id: "$category_code",
            count: { $sum: 1 }
        }
    },
    { $sort: { count: -1 }}
]).pretty()

In [ ]:
# $sortByCount single query facet for the previous search
db.companies.aggregate([
    {
        $match: {
            $text: { $search: "network" }
        }
    },
    { $sortByCount: "$category_code" }
]).pretty()

In [ ]:
# extend the pipeline for a more elaborate facet
db.companies.aggregate([
    {
        $match: {
            $text: { $search: "network" }
        }
    },
    {
        $unwind: "$offices"
    },
    {
        $match: { "offices.city": { $ne: "" }}
    },
    {
        $sortByCount: "$offices.city"
    }
]).pretty()

---

In [ ]:
# performing a group followed by a sort to rank occurence
db.movies.aggregate([
    {
        $group: {
            _id: "$imdb.rating",
            count: { $sum: 1}
        }
    },
    { $sort: { count: -1 }}
])

In [ ]:
# sortByCount is equivalent to the above. In fact, if you execute this pipeline
# with { explain: true } you will see that it is transformed to the above!
db.movies.aggregate([
    {
        $sortByCount: "$imdb.rating"
    }
])

# $bucket 的操作

In [ ]:
# create manual buckets using $ bucket
db.companies.aggregate([
    {
        $match: {
            founded_year: { $gt: 1980 },
            number_of_employees: { $ne: null }
        }
    },
    {
        $bucket: {
            groupBy: "$number_of_employees",
            boundaries: [ 0, 20, 50, 100, 500, 1000, Infinity]
        }
    }
])

In [ ]:
# reproduce error message for non matching documents
db.coll.insert({ x: "a" });
db.coll.aggregate([{ $bucket: {groupBy: "$x", boundaries: [0, 50, 100]}}])

In [ ]:
# set `default` option to collect documents that do not match boundaries
db.companies.aggregate([
    {
        $match: {
            founded_year: { $gt: 1980 }
        }
    },
    {
        $bucket: {
            groupBy: "$number_of_employees",
            boundaries: [ 0, 20, 50, 100, 500, 1000, Infinity],
            default: "Other"
        }
    }
])

In [ ]:
# reproduce error message for inconsitent boundaries datatype
db.coll.aggregate([{ $bucket: {groupBy: "$x", boundaries: ["a", "b", 100]}}])

In [ ]:
# set `output` option for $bucket stage
db.companies.aggregate([
    {
        $match: {
            founded_year: { $gt: 1980 }
        }
    },
    {
        $bucket: {
            groupBy: "$number_of_employees",
            boundaries: [ 0, 20, 50, 100, 500, 1000, Infinity],
            default: "Other",
            output: {
                total: { $sum: 1},
                average: { $avg: "$number_of_employees" },
                categories: { $addToSet: "$category_code" }
            }
        }
    }
]).pretty()

# $bucketAuto 的操作

In [ ]:
# generate buckets automatically with $bucktAuto stage
db.companies.aggregate([
    {
        $match: {
            "offices.city": "New York"
        }
    },
    {
        $bucketAuto: {
            groupBy: "$founded_year",
            buckets: 5
        }

    }
])

In [ ]:
# set `output` option for $bucketAuto
db.companies.aggregate([
    {
        $match: {
            "offices.city": "New York"
        }
    },
    {
        $bucketAuto: {
            groupBy: "$founded_year",
            buckets: 5,
            output: {
                total: { $sum: 1},
                average: { $avg : "$number_of_employees" } 
            }
        }

    }
])

In [ ]:
# default $buckeAuto behaviour
for(i=1; i <= 1000; i++) {  db.series.insert( {_id: i}  ) };

db.series.aggregate([
    {
        $bucketAuto: {
            groupBy: "$_id",
            buckets: 5
        }
    }
])

In [ ]:
# generate automatic buckets using granularity numerical series R20
db.series.aggregate([
    {
        $bucketAuto: {
            groupBy: "$_id",
            buckets: 5,
            granularity: "R20"
        }
    }
])

# $facet 的操作

In [ ]:
# render several different facets using $facet stage
db.companies.aggregate([
    {
        $match: {
            $text: { $search: "Databases" }
        }
    },
    {
        $facet: {
            Categories: [{ $sortByCount: "$category_code" }],
            Employees: [
                { $match: { founded_year: { $gt: 1980 }}},
                {
                    $bucket: {
                        groupBy: "$number_of_employees",
                        boundaries: [0, 20, 50, 100, 500, 1000, Infinity],
                        default: "Other"
                    }
                }
            ],
            Founded: [
                { $match: { "offices.city": "New York" }},
                {
                    $bucketAuto: {
                        groupBy: "$founded_year",
                        buckets: 5
                    }
                }
            ]

        }
    }
]).pretty()

### 範例01

In [ ]:
# 參考01
db.movies.aggregate([
    {
        $match: {
            metacritic: { $ne: null},
            "imdb.rating": { $gt: 0 }
        }
    },
    {
        $project:{
            title: 1,
            imdb_rating: "$imdb.rating",
            metacritic: 1
        }
    },
    {
        $facet: {
            imdbTop: [
                { $sort: { imdb_rating: -1 }},
                { $limit: 10 }
            ],
            metacriticTop: [
             { $sort: { metacritic: -1 }},
             { $limit: 10 }
            ]
        }
    },
    {
        $group: {
            _id: null,
            imdbTop: { $addToSet: "$imdbTop.title" },
            metacriticTop: { $addToSet: "$metacriticTop.title" }
        }
    },
    { $unwind: "$imdbTop" },
    { $unwind: "$metacriticTop" },
    {
        $project: {
            interSection: { $setIntersection: [ "$imdbTop", "$metacriticTop" ]}
        }
    }
]).pretty()

In [ ]:
# 參考02
db.movies.aggregate([
  {
    $match: {
      metacritic: { $gte: 0 },
      "imdb.rating": { $gte: 0 }
    }
  },
  {
    $project: {
      _id: 0,
      metacritic: 1,
      imdb: 1,
      title: 1
    }
  },
  {
    $facet: {
      top_metacritic: [
        {
          $sort: {
            metacritic: -1,
            title: 1
          }
        },
        {
          $limit: 10
        },
        {
          $project: {
            title: 1
          }
        }
      ],
      top_imdb: [
        {
          $sort: {
            "imdb.rating": -1,
            title: 1
          }
        },
        {
          $limit: 10
        },
        {
          $project: {
            title: 1
          }
        }
      ]
    }
  },
  {
    $project: {
      movies_in_both: {
        $setIntersection: ["$top_metacritic", "$top_imdb"]
      }
    }
  }
])